In [1]:
#coding=utf-8
import numpy as np
import PyKDL as kdl
import urdf_parser_py.urdf as urdf
import xml.dom.minidom
import os
import numpy as np
import copy

In [2]:
def get_urdf_parameter(joint_id,xml_obj):
    """ get inertial data. """
    # mass = xml_obj.getElementsByTagName("mass")[joint_id].getAttribute("value")
    rpy  = xml_obj.getElementsByTagName("joint")[joint_id].getElementsByTagName("origin")[0].getAttribute("rpy").split(" ")
    rpy  = tuple([float(i) for i in rpy])
    axis  = xml_obj.getElementsByTagName("joint")[joint_id].getElementsByTagName("axis")[0].getAttribute("xyz").split(" ")
    axis  = np.array(axis,dtype=np.float32)
    xyz  = xml_obj.getElementsByTagName("joint")[joint_id].getElementsByTagName("origin")[0].getAttribute("xyz").split(" ")
    xyz  = np.array(xyz,dtype=np.float32)
    data = [axis, rpy,xyz]
    return data

In [3]:
# robot = urdf.URDF.from_xml_file('dofbot.urdf')
xmlDoc = xml.dom.minidom.parse('dofbot.urdf')                 # 加载文件
link_count  = xmlDoc.getElementsByTagName("link").length   # 统计 link  数量
joint_count = xmlDoc.getElementsByTagName("joint").length  # 统计 joint 数量
# print(joint_count)
parameter_robot = {}
for i in range(joint_count):
    data = get_urdf_parameter(i,xmlDoc)
    print(data)
    parameter_robot[i]=data

[array([0., 0., 1.], dtype=float32), (0.0, 0.0, 0.0), array([0.   , 0.   , 0.064], dtype=float32)]
[array([0., 0., 1.], dtype=float32), (0.0, 1.5708, 0.0), array([0.    , 0.    , 0.0435], dtype=float32)]
[array([0., 0., 1.], dtype=float32), (0.0, 0.0, 0.0), array([-0.08285,  0.     ,  0.     ], dtype=float32)]
[array([0., 0., 1.], dtype=float32), (0.0, 0.0, 0.0), array([-0.08285,  0.     ,  0.     ], dtype=float32)]
[array([0., 0., 1.], dtype=float32), (0.0, -1.5708, 0.0), array([-0.07385, -0.00215,  0.     ], dtype=float32)]


In [4]:
parameter_robot[0]

[array([0., 0., 1.], dtype=float32),
 (0.0, 0.0, 0.0),
 array([0.   , 0.   , 0.064], dtype=float32)]

In [135]:
# import copy #用于深度拷贝
 
# 注意一些基本概念
# Segment初始化由绕的轴和末端姿态来生成，可能跟书上的D-H方法有点不一样，区别在于用末端和轴来定义一个关节
#   轴是在当前关节的坐标系中绕轴V旋转，默认设置为kdl.Joint.RotZ
#   f_tip，末端在当前坐标系中的姿态
 
jnts = []
frms = []
 
for i in range(joint_count):
    jnts.append(kdl.Joint(kdl.Joint.RotZ))
 
# 注意，这个框架是f_tip，末端
frame = []
for i in parameter_robot:
    frm = kdl.Frame(kdl.Rotation.RPY(parameter_robot[i][1][0],parameter_robot[i][1][1],parameter_robot[i][1][2]), kdl.Vector(parameter_robot[i][2][0],parameter_robot[i][2][1],parameter_robot[i][2][2]))
    frame.append(frm)
 
 

In [277]:
link = []
rbt = kdl.Chain() 
for i in range(joint_count):
    link.append(kdl.Segment(jnts[i], frame[i]))
    rbt.addSegment(link[i])
 
fk = kdl.ChainFkSolverPos_recursive(rbt)
 
p = kdl.Frame()
q = kdl.JntArray(5)
q_out = kdl.JntArray(5)

q[0] = 0
q[1] = 0
q[2] = 0
q[3] = 0
q[4] = 0


fk.JntToCart(q, p)

ik = kdl.ChainIkSolverPos_LMA(rbt,eps=float(1e-10), maxiter=int(1e8))

ik.CartToJnt(q,p,q_out)
print(p)

[[           1,           0,           0;
            0,           1,           0;
            0,           0,           1]
[ 8.79916e-07,    -0.00215,     0.34705]]


In [276]:
from tqdm import tqdm
import time
start_time = time.time()
is_success = []
for i in tqdm(range(int(1e3))):
    # Do some work
    
    q[0] = np.random.uniform(-np.pi/2,np.pi/2)
    q[1] = np.random.uniform(-np.pi/2,np.pi/2)
    q[2] = np.random.uniform(-np.pi/2,np.pi/2)
    q[3] = np.random.uniform(-np.pi/2,np.pi/2)
    q[4] = np.random.uniform(0,0.0)
    fk.JntToCart(q, p)
    q_int = kdl.JntArray(5)
    q_int[0] = q[0]
    q_int[1] = 0
    q_int[2] = 0
    q_int[3] = 0
    q_int[4] = q[4]
    # q_int = q
    temp = int(ik.CartToJnt(q_int,p,q_out))
    is_success.append(temp)
end_time = time.time()
succcuss_rate = 1- (np.count_nonzero(is_success))/len(is_success)
print("success rate is: ", succcuss_rate)
print("Elapsed time: ", end_time - start_time, "seconds")


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3721.14it/s]

success rate is:  0.999
Elapsed time:  0.2778902053833008 seconds
